In [ ]:
import tensorflow as tf
import numpy as np
from numpy import matlib
def innerPd(pt0, pt1):
    return tf.mul(pt0[:,0],pt1[:,0]) + tf.mul(pt0[:,1],pt1[:,1])
def projPt(lnPt0, lnPt1, pt):
    pt = tf.sub(pt , lnPt0)
    lnPt1 = tf.sub(lnPt1 , lnPt0)
    lnLen = tf.sqrt(innerPd(lnPt1, lnPt1))
    coefs3 = innerPd(pt, lnPt1)
    coefs = tf.mul( tf.sign(coefs3) , tf.sqrt( tf.abs(coefs3) ) )
    coefs2 = tf.div(coefs, lnLen)
    dup1 = tf.pack([coefs2, coefs], axis = 1)
    ppp = tf.mul( dup1  , lnPt1 /2) 
    pjs = lnPt0 + ppp
    return pjs
def distLmtd(lnPt0, lnPt1, pt):
    pj = projPt(lnPt0, lnPt1, pt)
    
    dist1 = tf.sub(pt[0] , pj[:, 0])
    dist1 = tf.pow(dist1, 2)
    
    dist2 = tf.sub(pt[1] , pj[:, 1])
    dist2 = tf.pow(dist2, 2)
    
    dist = tf.pow(tf.add(dist1, dist2), 0.5)
    
    return (dist, pj)
def closeTo(allMat, sign):
    filter_ = tf.ones( [allMat.shape[0]])
    filter_ = tf.equal(filter_, filter_ )
    idRules = [[0,0],[0,2],[1,1],[1,3]]
    for rule in idRules:
        t1 = allMat[:, rule[1]] - sign[ rule[0] ]
        t2 = tf.less( tf.abs( t1 ), 0.004 )
        filter_ = tf.logical_and( filter_ , t2 )
    inds_ = tf.where(filter_)

    allMatTf = tf.constant(allMat)
    ged_ = tf.gather(allMatTf, inds_)

    ged_ = tf.reshape(ged_, [-1,4])
    return ged_
def findClosest(allPairsMat_, sign):
    allPairsMat_ = closeTo(allPairsMat_, sign)
    dists, pjs = distLmtd( allPairsMat_[:,0:2], allPairsMat_[:,2:4], sign )
    minIdx = tf.argmin(dists, 0)
    minDist = tf.gather(dists, minIdx)
    minPair = tf.gather(allPairsMat_, minIdx)
    minPjs = tf.gather(pjs, minIdx)
    return ( minDist, minPair, minPjs, pjs, dists, allPairsMat_ )

allPairsMat_ = np.load('allPairsMat.npy')
def calcFunc(sign):
    dist_, line_, _, _, _, _ = findClosest(allPairsMat_, sign)
    return [dist_, line_[0], line_[1], line_[2], line_[3] ]

import numpy as np
import pandas as pd

signals = pd.read_csv('signalisation.csv')
signalCoordinates = np.matrix( [signals['Latitude'].values, signals['Longitude'].values] ).T

def processJobAllS(jobAllS):
    res = tf.map_fn( calcFunc , jobAllS, dtype=[tf.float64, tf.float64, tf.float64, tf.float64, tf.float64] )
    return res

in_vals = signalCoordinates[0:100, :]

in_ = tf.placeholder(tf.float64, in_vals.shape)
batchsize = int(in_vals.shape[0] / 2)
with tf.device("/job:tf/task:0"):
    in_1 = in_[0:batchsize]
    out_1 = processJobAllS(in_1)

with tf.device("/job:tf/task:0"):
    in_2 = in_[batchsize:]
    out_2 = processJobAllS(in_2)
    out_ = tf.pack([out_1, out_2], axis = 0)
    

with tf.Session("grpc://tensorflow2:2222") as sess:
    out_vals = sess.run(out_, feed_dict={in_:in_vals})
    
#print(in_vals)
#print(out_vals[0].T)
print(out_vals.shape)

In [2]:
finalres = np.zeros([in_vals.shape[0], 5])
batchSize = out_vals[0].shape[1]
for i in range(out_vals.shape[0]):
    finalres[batchSize*i:batchSize*(i+1), :] = np.asmatrix(out_vals[i].T)

import folium
%matplotlib inline
map_ = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
tt = finalres
for i in range(finalres.shape[0]):

    signal_coor = (in_vals[i,0], in_vals[i,1])
    
    line = [(tt[i, 1], tt[i, 2]), (tt[i, 3], tt[i, 4])]
    
    folium.Marker(signal_coor, popup = 'sign' + str(i), icon = folium.Icon(color ='blue')).add_to(map_)
    folium.PolyLine( line , color="red", weight=2.5, opacity=1).add_to(map_)
#map_


<img src="./closestSegments.bmp">